In [1]:
import torch
import torch.nn as nn
import os
import scipy
import numpy as np
import pandas as pd
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import r2_score
from PIL import Image
from torch.hub import load_state_dict_from_url
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# 图片路径
image_folder = "E:\shujuji\metamaterialpicture9001features"
# Excel文件路径
excel_path = "E:\\1-1000-90.xlsx"

In [ ]:
df = pd.read_excel(excel_path, usecols=[0, 16])  # 使用第1列和第17列的数据，从0开始计数，并跳过第一行

In [ ]:
# 初始化图像和性能数组
X = []
y = []

In [ ]:
for i in range(1, 651):
    img_path = os.path.join(image_folder, f"lcy{i}.jpg")
    if i in df["编号"].values:
        img = Image.open(img_path).convert("L")  # Open image and convert to grayscale
        img = img.resize((20, 20))  # 调整为合适的图像大小
        img_array = np.array(img)
        X.append(img_array)
X = np.array(X)
y = df["d33"].to_numpy()

In [ ]:
# 图片路径
image_folder1 = "E:\metamaterialpicture\metamaterialpicture9001features"
# Excel文件路径
excel_path1 = "E:\metamaterialpicture\metamaterialpicture90\\1-430-90.xlsx"
df1 = pd.read_excel(excel_path1, usecols=[0, 13])  # 使用第1列和第17列的数据，从0开始计数，并跳过第一行
# 初始化图像和性能数组
X1 = []
y1 = []
for i in range(1, 431):
    img_path = os.path.join(image_folder1, f"lcy{i}.jpg")
    if i in df1["编号"].values:
        img = Image.open(img_path).convert("L")  # Open image and convert to grayscale
        img = img.resize((20, 20))  # 调整为合适的图像大小
        img_array = np.array(img)
        X1.append(img_array)

y1=df1["d33"]
# 转换为NumPy数组
X1 = np.array(X1)
y1 = np.array(y1)

In [ ]:
# 图片路径
image_folder2 = "E:\metamaterialpicture\me901features"
# Excel文件路径
excel_path2 = "E:\metamaterialpicture\me90\\1-300-90.xlsx"
df2 = pd.read_excel(excel_path2, usecols=[0, 14])  # 使用第1列和第17列的数据，从0开始计数，并跳过第一行
# 初始化图像和性能数组
X2 = []
y2 = []
for i in range(1, 302):
    img_path = os.path.join(image_folder2, f"lcy{i}.jpg")
    if i in df2["编号"].values:
        img = Image.open(img_path).convert("L")  # Open image and convert to grayscale
        img = img.resize((20, 20))  # 调整为合适的图像大小
        img_array = np.array(img)
        X2.append(img_array)

y2=df2["d33"]
# 转换为NumPy数组
X2 = np.array(X2)
y2 = np.array(y2)

In [ ]:
X3=np.concatenate((X, X1), axis=0)
y3=np.concatenate((y, y1), axis=0)
X4=np.concatenate((X3, X2), axis=0)
y4=np.concatenate((y3, y2), axis=0)
X_train, X_test, y_train, y_test = train_test_split(X4, y4, test_size=0.2, random_state=500)
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')
print(y_train.shape)

In [ ]:
##resnet每个残差链接模块
class BasicBlock(nn.Module):
    def __init__(self,inplanes: int,planes: int,stride: int = 1,downsample = None) -> None:
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes,planes, 3, stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, 3, stride=1,padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride


    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out


class ResNet(nn.Module):

    def __init__(self,layers: list, num_classes: int,zero_init_residual: bool = False) -> None:
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.dilation = 1
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool1 = nn.AvgPool2d(kernel_size=3, stride=1, padding=1)

        self.layer1 = self._make_layer(64, layers[0], stride=1)
        self.layer2 = self._make_layer(128, layers[1], stride=2)
        self.layer3 = self._make_layer(256, layers[2], stride=2)
        self.layer4 = self._make_layer(512, layers[3], stride=2)
        self.avgpool2 = nn.AdaptiveAvgPool2d((1, 1))
        self.do = nn.Dropout(0.2)
        self.fc = nn.Linear(512, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)


    def _make_layer(self, planes: int, blocks: int,stride: int = 1) -> nn.Sequential:
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(nn.Conv2d(self.inplanes, planes, 1, stride=stride, bias=False),
                                       nn.BatchNorm2d(planes))
        layers = []
        layers.append(BasicBlock(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        #先做7x7的卷积
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.avgpool1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool2(x)
        x = torch.flatten(x, 1)
        x = self.do(x)
        x = self.fc(x)

        return x


In [ ]:
# 创建ResNet-18模型
model=ResNet([2, 2, 2, 2], 1, zero_init_residual= True)
# 编译模型，选择合适的优化器和损失函数
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=1e-2)
# 为训练集和验证集定义 DataLoader
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_test, y_test)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


# 存储训练集和测试集的损失和R²评分
train_losses = []
train_r2_scores = []
test_losses = []
test_r2_scores = []

# 训练循环
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    predictions_train = []
    targets_train = []

    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(1), targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # 记录训练集的预测值和目标值
        predictions_train.extend(outputs.squeeze(1).detach().numpy())
        targets_train.extend(targets.numpy())

    # 计算训练集的 R²评分
    train_r2 = r2_score(targets_train, predictions_train)
    train_losses.append(total_loss / len(train_loader))
    train_r2_scores.append(train_r2)

    # 测试
    test_loss = 0.0
    predictions_test = []
    targets_test = []

    for inputs, targets in val_loader:
        outputs = model(inputs)
        outputs = outputs.squeeze(1)
        test_loss += criterion(outputs, targets).item()

        # 记录测试集的预测值和目标值
        predictions_test.extend(outputs.detach().numpy())
        targets_test.extend(targets.numpy())

    # 计算测试集的 R²评分
    test_r2 = r2_score(targets_test, predictions_test)
    test_losses.append(test_loss / len(val_loader))
    test_r2_scores.append(test_r2)

    # 输出训练集和测试集的损失和R²评分
    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_losses[-1]:.4f}, Train R²: {train_r2_scores[-1]:.4f}, '
          f'Test Loss: {test_losses[-1]:.4f}, Test R²: {test_r2_scores[-1]:.4f}')


In [ ]:
# 绘制损失和R²评分的图表
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_r2_scores, label='Train R²')
plt.plot(test_r2_scores, label='Test R²')
plt.xlabel('Epoch')
plt.ylabel('R² Score')
plt.legend()

plt.show()